# Preprocessing Humidity and Windspeed Data

## Import some libraries

In [2]:
import pandas as pd
import numpy as np
from pandas.tseries.offsets import DateOffset
from pandas import Timestamp

## Import Data

In [2]:
weather_data = pd.read_csv("humidity_windspeed_data/extended_humidity_windspeed.csv")

In [3]:
weather_data

,Day,Month,Year,Humidity,State,Suburb,location,Windspeed,Date Time
0,1,12,2008,71.0,VIC,Ballarat,Ballarat North,26,2008-12-01
1,2,12,2008,66.0,VIC,Ballarat,Ballarat North,20,2008-12-02
2,3,12,2008,82.0,VIC,Ballarat,Ballarat North,22,2008-12-03
3,4,12,2008,60.0,VIC,Ballarat,Ballarat North,9,2008-12-04
4,5,12,2008,74.0,VIC,Ballarat,Ballarat North,30,2008-12-05
...,...,...,...,...,...,...,...,...,...
41719,22,9,2020,70.0,VIC,NaN,Horsham,39,2020-09-22
41720,23,9,2020,81.0,VIC,NaN,Horsham,24,2020-09-23
41721,24,9,2020,95.0,VIC,NaN,Horsham,19,2020-09-24
41722,25,9,2020,79.0,VIC,NaN,Horsham,24,2020-09-25


Count number of missing values of the humidity

In [4]:
len(weather_data[weather_data["Humidity"].isnull()==True])

6136

Convert the Date Time column from string to Timestamp type

In [5]:
weather_data["Date Time"]=pd.to_datetime(weather_data["Date Time"])
temp_weather_data = weather_data.copy()

Find the starting date and ending date recording weather data for each LGA

In [6]:
start_date=weather_data.groupby('location',as_index=False)['Date Time'].min()
end_date=weather_data.groupby('location',as_index=False)['Date Time'].max()

### Find the missing value

Calculate the mean of humidity

In [7]:
missing_humidity_data = weather_data[np.isnan(weather_data['Humidity'])][['Date Time','location']]

In [8]:
humidity_mean=weather_data.describe().loc['mean','Humidity']

## Fill the missing data for humidity

In [11]:
# Go through each row of the missing_humidity_data Dataframe
for index, row in missing_humidity_data.iterrows():
    date,location = row
    #if the year is the starting year, we look up humidity data in the next years 2008, 2009,... (until we find not-null value)
    if date.strftime('%Y')==pd.to_datetime(start_date.loc[start_date['location']==location,'Date Time'].values[0]).strftime('%Y'):
        date_offset=date + DateOffset(years=1)
        find_notnull=False
        while date_offset<=end_date.loc[end_date['location']==location,'Date Time'].values[0]:
            nextyear_humidity9am = temp_weather_data.loc[(temp_weather_data['Date Time']==date_offset) & (temp_weather_data['location']== location),'Humidity'].values[0]
            #If we find a not null humidity
            if np.isnan(nextyear_humidity9am)==False :
                temp_weather_data.at[index,'Humidity'] = nextyear_humidity9am
                find_notnull=True
                break
            date_offset=date_offset + DateOffset(years=1)
        #if we cannot find not null value
        if find_notnull==False:
            temp_weather_data.at[index,'Humidity'] = humidity_mean
            
    #if the year is upper limit year, we look up humidity data in the previous years 2016, 2015,...(until we find not-null value)
    elif date.strftime('%Y')==pd.to_datetime(end_date.loc[end_date['location']==location,'Date Time'].values[0]).strftime('%Y'):
        date_offset=date - DateOffset(years=1)
        find_notnull=False
        while date_offset>=pd.to_datetime(start_date.loc[start_date['location']==location,'Date Time'].values[0]):
            lastyear_humidity9am =  temp_weather_data.loc[(temp_weather_data['Date Time']==date_offset) & (temp_weather_data['location']== location),'Humidity'].values[0]
            #If we find a not null humidity
            if np.isnan(lastyear_humidity9am)==False:
                temp_weather_data.at[index,'Humidity'] = lastyear_humidity9am
                find_notnull=True
                break
            date_offset=date_offset - DateOffset(years=1)
        #if we cannot find not null value
        if find_notnull==False:
            temp_weather_data.at[index,'Humidity'] = humidity_mean
    #if the year is between the starting year and ending year 
    else:
        filling_missing=0
        date_offset=date + DateOffset(years=1)
        nextyear_humidity9am=-1
        #Find the next year humidity on the same day
        while date_offset<=pd.to_datetime(end_date.loc[end_date['location']==location,'Date Time'].values[0]):
            nextyear_humidity9am = temp_weather_data.loc[(temp_weather_data['Date Time']==date_offset) & (temp_weather_data['location']== location),'Humidity'].values[0]
    
            if np.isnan(nextyear_humidity9am)==False:
                break
            date_offset=date_offset + DateOffset(years=1)
        date_offset= date-DateOffset(years=1)
        lastyear_humidity9am=-1
        #Find the previous humidity on the same day
        while date_offset>=pd.to_datetime(start_date.loc[start_date['location']==location,'Date Time'].values[0]):
            lastyear_humidity9am =  temp_weather_data.loc[(temp_weather_data['Date Time']==date_offset) & (temp_weather_data['location']== location),'Humidity'].values[0]
            if np.isnan(lastyear_humidity9am)==False:
                break
            date_offset=date_offset - DateOffset(years=1)
        #Check if previous and next humidity on the same day are not null
        #If these values are null then fill by mean humidity
        if np.isnan(nextyear_humidity9am) == True and np.isnan(lastyear_humidity9am)==True:
            filling_missing = humidity_mean
        #If one of these values is null
        elif np.isnan(nextyear_humidity9am) == True and np.isnan(lastyear_humidity9am)==False:
            filling_missing = lastyear_humidity9am
        elif np.isnan(nextyear_humidity9am) ==False and np.isnan(lastyear_humidity9am)==True:
            filling_missing = nextyear_humidity9am
        #If both of them are not null
        else:
            filling_missing = (nextyear_humidity9am + lastyear_humidity9am)/2
        temp_weather_data.loc[(temp_weather_data['Date Time']==date) & (temp_weather_data['location']== location),'Humidity'] = filling_missing

Check the null value after filling the missing data

In [13]:
len(temp_weather_data[temp_weather_data["Humidity"].isnull()==True])

0

Write to file

In [15]:
temp_weather_data.to_csv("humidity_windspeed_data/processed_humidity_windspeed_data.csv",index=False)

## Preprocessing Wind speed Data

Read data from csv file

In [18]:
weather_data=pd.read_csv("humidity_windspeed_data/processed_humidity_windspeed_data.csv")
weather_data["Date Time"]=pd.to_datetime(weather_data["Date Time"])


Convert the data type to integer

In [19]:
weather_data["Windspeed"]=pd.to_numeric(temp_weather_data["Windspeed"], errors='coerce')
temp_weather_data = weather_data.copy()

Count the missing values

In [20]:
len(weather_data[weather_data["Windspeed"].isnull()==True])

8018

In [21]:
missing_windpseed_data = weather_data[weather_data["Windspeed"].isnull()==True][['Date Time','location']]

In [22]:
missing_windpseed_data

,Date Time,location
34,2009-01-04,Ballarat North
109,2009-03-20,Ballarat North
150,2009-04-30,Ballarat North
192,2009-06-11,Ballarat North
198,2009-06-17,Ballarat North
...,...,...
41664,2020-07-29,Horsham
41665,2020-07-30,Horsham
41669,2020-08-03,Horsham
41672,2020-08-06,Horsham


Find the starting data and ending date

In [23]:
start_date=weather_data.groupby('location',as_index=False)['Date Time'].min()
end_date=weather_data.groupby('location',as_index=False)['Date Time'].max()

In [24]:
windspeed_mean=weather_data.describe().loc['mean','Windspeed']

## Fill the missing windspeed

In [25]:
# Go through each row of the missing_windspeed_data Dataframe
for index, row in missing_windpseed_data.iterrows():
    date,location = row
    #if the year is the start year, we look up windspeed data in the next years 2008, 2009,... (until we find not-null value)
    if date.strftime('%Y')==pd.to_datetime(start_date.loc[start_date['location']==location,'Date Time'].values[0]).strftime('%Y'):
        date_offset=date + DateOffset(years=1)
        find_notnull=False
        while date_offset<=end_date.loc[end_date['location']==location,'Date Time'].values[0]:
            nextyear_windspeed9am = temp_weather_data.loc[(temp_weather_data['Date Time']==date_offset) & (temp_weather_data['location']== location),'Windspeed'].values[0]
            #If we find a not null windspeed
            if pd.isnull(nextyear_windspeed9am)==False :
                #Assign that value to the current missing value
                temp_weather_data.at[index,'Windspeed'] = nextyear_windspeed9am
                find_notnull=True
                break
            date_offset=date_offset + DateOffset(years=1)
        #if we cannot find not null value
        if find_notnull==False:
            #Assign to the mean wind speed
            temp_weather_data.at[index,'Windspeed'] = windspeed_mean
            
    #if the year is upper limit year, we look up humidity data in the previous years 2016, 2015,...(until we find not-null value)
    elif date.strftime('%Y')==pd.to_datetime(end_date.loc[end_date['location']==location,'Date Time'].values[0]).strftime('%Y'):
        date_offset=date - DateOffset(years=1)
        find_notnull=False
        while date_offset>=pd.to_datetime(start_date.loc[start_date['location']==location,'Date Time'].values[0]):
            lastyear_windspeed9am =  temp_weather_data.loc[(temp_weather_data['Date Time']==date_offset) & (temp_weather_data['location']== location),'Windspeed'].values[0]
            #If we find a not null wind speed
            if pd.isnull(lastyear_windspeed9am)==False:
                #Assign that value to the current missing value
                temp_weather_data.at[index,'Windspeed'] = lastyear_windspeed9am
                find_notnull=True
                break
            date_offset=date_offset - DateOffset(years=1)
        #if we cannot find not null value
        if find_notnull==False:
            #Assign to the mean wind speed value
            temp_weather_data.at[index,'Windspeed'] = windspeed_mean
    #If the year of current missing value is between starting year and ending year
    else:
        filling_missing=0
        date_offset=date + DateOffset(years=1)
        nextyear_windspeed9am=-1
        #Find the next year wind speed on the same day
        while date_offset<=pd.to_datetime(end_date.loc[end_date['location']==location,'Date Time'].values[0]):
            nextyear_windspeed9am = temp_weather_data.loc[(temp_weather_data['Date Time']==date_offset) & (temp_weather_data['location']== location),'Windspeed'].values[0]
    
            if pd.isnull(nextyear_windspeed9am)==False:
                break
            date_offset=date_offset + DateOffset(years=1)
        date_offset= date-DateOffset(years=1)
        lastyear_windspeed9am=-1
        #Find the previous year wind speed on the same day
        while date_offset>=pd.to_datetime(start_date.loc[start_date['location']==location,'Date Time'].values[0]):
            lastyear_windspeed9am =  temp_weather_data.loc[(temp_weather_data['Date Time']==date_offset) & (temp_weather_data['location']== location),'Windspeed'].values[0]
            if pd.isnull(lastyear_windspeed9am)==False:
                break
            date_offset=date_offset - DateOffset(years=1)
        #Check if the wind speed value on the same day of the previous year and next year are not null
        #If both are null then fill by the mean of windspeed
        if pd.isnull(nextyear_windspeed9am) == True and pd.isnull(lastyear_windspeed9am)==True:
            filling_missing = windspeed_mean
        #If one of these values is null
        elif pd.isnull(nextyear_windspeed9am) == True and pd.isnull(lastyear_windspeed9am)==False:
            filling_missing = lastyear_windspeed9am
        elif pd.isnull(nextyear_windspeed9am) ==False and pd.isnull(lastyear_windspeed9am)==True:
            filling_missing = nextyear_windspeed9am
        #If both are not null
        else:
            filling_missing = (nextyear_windspeed9am + lastyear_windspeed9am)/2
        temp_weather_data.loc[(temp_weather_data['Date Time']==date) & (temp_weather_data['location']== location),'Windspeed'] = filling_missing

Count the missing data after filling the missing data

In [27]:
len(temp_weather_data[temp_weather_data["Windspeed"].isnull()==True])

0

Write to file

In [33]:
temp_weather_data["Windspeed"]=np.ceil(temp_weather_data["Windspeed"]).astype(int)

In [34]:
temp_weather_data.to_csv("humidity_windspeed_data/processed_humidity_windspeed_data.csv",index=False)